# Part 4. Model-based analysis

In the previous part, we pre-processed the fmri data, we fit the behavioral model, and extracted signal from each participant's striatum. Now, we're ready to do the final step: the model-based analysis, to see if differences in signal in striatum due to a `speed` cue versus an `accuracy` cue correlate with threshold differences.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy as sp
from scipy import stats

import glob
import re
%matplotlib inline

We start out by loading all extracted striatum signal of all participants. Using the following line of code, you can make a list of all `.txt`-files that were generated before.

In [ ]:
fns = glob.glob('/data/extracted_signals/extracted_timeseries/_mask_*_subject_id_*/_extract*/*.txt')
fns[:10]

Each of the file names contains the subject id, the applied mask, and the run/block number. We later want to extract these, and for this we can use a regular expression. It's a bit of an 'art' to create these, but you want something like this:

In [ ]:
reg = re.compile('.*/_mask_(?P<mask_name>.*)_subject_id_(?P<subj_idx>.*)/_extract_mean_ts[0-9]/sub-[0-9]*_task-SAT_run-(?P<block>[0-9])_bold_space-MNI152NLin2009cAsym_preproc_ts.txt')

To visualize what reg does, you can do the following:

In [ ]:
for fn in fns[:5]:
    print(fn)
    print(reg.match(fn).groupdict())

1) What does the call `reg.match(fn).groupdict()` do?

An especially useful library is [pandas](http://pandas.pydata.org/), which allows you to manipulate data in an `R`-like DataFrame. Next, I'll combine all neural data of all subjects in a single DataFrame.

In [ ]:
# create an empty list first
df = []

# loop over .txt-files, adding signal row-by-row
for fn in fns:
    d = reg.match(fn).groupdict()  # here, we extract the paths
    d['signal'] = np.loadtxt(fn)
    df.append(d)

df = pd.DataFrame(df)  # here, we convert the list to a DataFrame
df['block'] = df['block'].astype(int)
df['subj_idx'] = df['subj_idx'].astype(int)

2) Have a look at the head (first few lines) of the dataframe, what do you see? Can you plot the signal for a few subjects?

In [ ]:
df.head()

### A look at design matrices
Crucial to the GLM is the design matrix. The general idea is that you, as a researcher, specify when in the experiment each 'event' took place (i.e., each cue, stimulus, perhaps responses - anything that you think may elicit a BOLD response). Then, you convolve these events with the canonical hemodynamic response function. The result of this convolution is the _predicted_ timeseries.

`Nipy` is a Python package that allows you to easily create a design matrix. Before turning to the actual experiment, let's make a dummy design matrix

In [ ]:
# Import some functions
from nipy.modalities.fmri import design_matrix, experimental_paradigm

In [ ]:
# Suppose we create a dummy experiment with 2 conditions. Each condition occurs three times in the experiment
conditions = ['condition1'] * 3 + ['condition2'] * 3

# Then, we specify *when* (time in seconds from experiment onset, start counting at 0) events took place (collapse over conditions here)
onsets = [5, 25 , 35, 15, 30, 40]

# You need to know the TR of the scanning sequence
tr = 2.0

# Create a vector containing all the time points at which you have a volume (i.e. 'scan'/'image')
frametimes = np.arange(0, 50, tr)

# Define the type of hemodynamic response function (hrf) you want to use
hrf_model = 'Canonical'

# Set-up the paradigm
paradigm =  experimental_paradigm.BlockParadigm(con_id=conditions, 
                                                onset=onsets,
                                                duration=[[1.]] * len(conditions))  
# Note that duration here is *not* the TR, but the duration of the events

X, names = design_matrix.dmtx_light(frametimes, paradigm, hrf_model=hrf_model, hfcut=100)

3) Have a look at the newly created variable X. What is its shape (how many columns & rows are there)? Also plot X. What are each of the lines?

In [ ]:
X.shape

In [ ]:
plt.plot(X)

## Let's make the real design matrices
In BIDS-format, the information about the events in the experiment is always placed together with the BOLD-nifti. That is, you can find it here:

`/data/bids/sub-<subject_id>/func/sub-<subject_id>_task-SAT_run-<run_idx>_events.tsv'`  

(.tsv is a tab-separated file, very much like a csv)

Let's load the events of a single subject (let's take 548) and a single run/block (block 1)

In [ ]:
subj_idx = 548
block = 1
mask = 'STR_L'

events = pd.read_csv('/data/bids/sub-%d/func/sub-%d_task-SAT_run-%d_events.tsv' %(subj_idx, subj_idx, block), sep='\t')
events = events[pd.notnull(events.event_type)]  # remove null trials

events.head()

`events` is now a DataFrame with four columns: onset (when in the experiment the event started), duration, weight (you can forget about this for this experiment, but this is what you would adjust if you have a parametric design), and event_type.

There are 4 event types in this run: a speed cue, an acc(uracy) cue, stimulus left, stimulus right. All events took 2 seconds. Now, turning this into a design matrix...

In [ ]:
conditions = events.event_type.tolist()
onsets = events.onset.tolist()
durations = events.duration.tolist()

tr = 1.994
frametimes = np.arange(0, 706, tr)
hrf_model = 'Canonical'


paradigm =  experimental_paradigm.BlockParadigm(con_id=conditions, 
                                                onset=onsets,
                                                duration=[[2.]] * len(conditions))

# Set up GLM
X, names = design_matrix.dmtx_light(frametimes, paradigm, hrf_model=hrf_model, drift_model='blank')
X = pd.DataFrame(X, columns=names)

Again, plot X. What are the 'conditions' here, and what are the 'onsets'?

In [ ]:
plt.plot(X)

# this line below re-sizes the plot to aid visualization
plt.gcf().set_size_inches((20, 5))

X is what we _expect_ a voxel does *if* it responds to the defined events. Now, in order to assess whether or not striatum responds to these events, we fit the data to our GLM, as follows:

In [ ]:
import statsmodels.api as sm
Y = df[(df.subj_idx == subj_idx) & (df.block == block) & (df.mask_name == mask)].iloc[0].signal
model = sm.OLS(Y, X)
r = model.fit()
r.summary()

5) Interpret these results

Now, we want to fit the GLM for all participants & blocks & masks. In the following function, we create the design matrix for each subject/mask/block, and fit the GLM.

6) Create a function that receives the subj_idx, block, and mask as arguments, and returns the beta-values of a fitted glm. (Most of this is copying the previous cells)

In [ ]:
def fit_glm(subj_idx, block, mask):
    
    # your code here. This is a bit tricky if you've never coded before, so you may want to peek at the answers

If your function is correct, running the following cell should give the beta values for acc (0.4672429), speed (1.563291), and a constant (389.445299)

In [ ]:
fit_glm(548, 1, 'STR_R')

In [ ]:
results = []

for fn in fns:
    subj_idx = int(reg.match(fn).groupdict()['subj_idx'])
    block = int(reg.match(fn).groupdict()['block'])
    for mask in ['STR_L', 'STR_R']:
        r = fit_glm(subj_idx, block, mask)
        
        for condition in ['acc', 'speed']:
            d = {}
            d['subj_idx'] = subj_idx
            d['mask_name'] = mask
            d['block'] = block
            d['condition'] = condition
            d['value'] = r[condition]
            results.append(d)

results = pd.DataFrame(results)

Have a look at the `results` DataFrame. 

In [ ]:
results

Alright, in the above cells we fitted a GLM for each subject & block & mask combination. Next, we want to combine these results with the behavioral results. In order to do this, it's useful to first transform the shape of the data frame a little bit.

In [ ]:
neural_results = results.pivot_table(values='value', columns=['condition', 'mask_name'], index=['subj_idx'])
neural_results

7) What did the cell above do?

Now we need to load the behavioral parameters

In [ ]:
behavioral_results = pd.read_csv('/data/behavior_fits/parameters_per_subject.csv')
behavioral_results.rename(columns={'Unnamed: 0': 'subj_idx'}, inplace=True)

# Get only threshold
behavioral_results = behavioral_results[['subj_idx', 'a.spd', 'a.acc']]
behavioral_results = behavioral_results.set_index('subj_idx')
behavioral_results

We can combine the behavioral & neural data as follows

In [ ]:
combined_data = pd.merge(neural_results, behavioral_results, suffixes=('_fmri', '_threshold'), left_index=True, right_index=True)
combined_data

8) Can you plot the a scatterplot with:
- on the x-axis the difference between threshold in accuracy and speed trials? (acc - speed)
- on the y-axis the difference in striatum activation after an accuracy cue and a speed cue (acc - speed);

Do this for left & right striatum separately.
One function you could use is seaborn's [jointplot](https://seaborn.pydata.org/generated/seaborn.jointplot.html)

In [ ]:
# your code here for right striatum

In [ ]:
# your code here for left striatum

What do you see? Is there a relation between striatal activation and threshold setting?

## You've reached the end